In [1]:
#Scrapes channel subscription data from Twitchtracker
import urllib.request as urllib
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from time import sleep
from random import randint

import pandas as pd

#Pull our existing data
ag_df = pd.read_csv("../../generated_data/TotalChannelData.csv")

channels = ag_df["Channel"].unique()
sub_df = pd.DataFrame()

for channel in channels:
    #bypass 403 forbidden against web crawlers
    url = "https://twitchtracker.com/" + channel + "/subscribers"

    print("Processing: " + channel)
    req = Request(url, headers={'User-Agent': 'Chrome/51.0.2704.103'})
    html = urlopen(req).read()

    soup = BeautifulSoup(html)

    table = soup.find(id="subscribers");
    #Only pull data for channels that do have subscribers
    if table is not None:
        rows = table.find_all("tr");
        for row in rows:
            data = row.find_all("td");
            #Knowing the structure of the table, process all valid rows in standard order
            #Append each processed data item to our dataframe
            if len(data) != 0:
                month = data[0].get_text().strip()
                total = data[1].get_text().strip()
                prime = data[2].get_text().strip()
                Tier1 = data[3].get_text().strip()
                Tier2 = data[4].get_text().strip()
                Tier3 = data[5].get_text().strip()
                Unshared = data[6].get_text().strip()
                # Income calculate the total dollar amounts based on the subscriber payments of the Tiers 
                Income = (int(Tier1)*4.99) + (int(Tier2)*9.99) + (int(Tier3)*24.99)

                data_dict = {"Channel": channel, "Month" : month, "Total": total, 
                             "Prime": prime, "Tier 1": Tier1, "Tier 2": Tier2, "Tier 3": Tier3, 
                             "Unshared": Unshared, "Total Income": '${:,.2f}'.format(Income)}
                sub_df = sub_df.append(data_dict, ignore_index=True)
    #bypass timeout against web crawlers (2-3 minutes for safety, total expected time ~2 hours)
    sleep(randint(120, 180))

sub_df.to_csv("../../generated_data/channelsubdata.csv")
sub_df



Processing: AdmiralBahroo


KeyboardInterrupt: 